## Import

In [10]:
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn

import pandas as pd

import os

import numpy as np

from skorch import NeuralNetRegressor
from skorch.callbacks import EarlyStopping, Checkpoint, LRScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from skorch.helper import predefined_split
from skorch.dataset import Dataset

from models import FFNeuralNetwork, LSTMNeuralNetwork, LSTMDataset
from utilities import create_scaled_data_by_col, rmsle

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Loading Data and Scaling

In [11]:
data_dir = 'data/'
df = pd.read_csv(os.path.join(data_dir, 'train_data.csv'))
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(by=['date', 'store_nbr'])
display(df.head())

,date,store_nbr,family,sales,onpromotion,city,state,store_type,cluster,oil,...,prev_1_sales,prev_7_sales,prev_14_sales,prev_1_transactions,prev_7_transactions,prev_14_transactions,is_payday,is_earth_quake,is_weekday,is_weekend
0,2013-02-01,1,0,3.0,0,0,0,0,13,97.46,...,0.0,0.0,0.0,0.0,0.0,0.0,False,False,1,0
1,2013-02-01,1,1,0.0,0,0,0,0,13,97.46,...,0.0,0.0,0.0,0.0,0.0,0.0,False,False,1,0
2,2013-02-01,1,2,0.0,0,0,0,0,13,97.46,...,0.0,0.0,0.0,0.0,0.0,0.0,False,False,1,0
3,2013-02-01,1,3,941.0,0,0,0,0,13,97.46,...,0.0,0.0,0.0,0.0,0.0,0.0,False,False,1,0
4,2013-02-01,1,4,0.0,0,0,0,0,13,97.46,...,0.0,0.0,0.0,0.0,0.0,0.0,False,False,1,0


In [12]:
min_max_cols = ['store_nbr', 'city', 'state', 'store_type', 'cluster', 'h_type_nat', 'h_description_nat', 'h_transferred_nat',
                'h_type_loc', 'h_description_loc', 'h_transferred_loc', 'month', 'day', 'day_of_week', 'is_payday',
                'is_earth_quake', 'is_weekday', 'is_weekend'
                ]
normalize_cols = ['onpromotion', 'oil', 'dow_avg_sales', 'dow_rolling_1_sales', 'dow_rolling_3_sales', 'prev_1_sales',
                'prev_7_sales', 'prev_14_sales', 'dow_avg_transactions', 'dow_rolling_1_transactions',
                'dow_rolling_3_transactions', 'prev_1_transactions', 'prev_7_transactions', 'prev_14_transactions'
                ]
x_cols = min_max_cols + normalize_cols
y_cols = ['sales']
split_col = 'family'

print(min_max_cols)
print(df.columns)

final_run = False

if final_run:
    train_df = df
else:
    rows_before = (df['date'] < '2017-08-01')
    rows_after = ~rows_before

    print('rows_before', rows_before.sum())
    print('rows_after', rows_after.sum())
    print('rows_total', len(df))

    train_df = df[rows_before]
    val_df = df[rows_after]

train_df_by_cluster = {}
scaler_x_by_cluster = {}
scaler_y_by_cluster = {}

for cluster in df[split_col].unique():
    cluster_df, cluster_min_max_scaler, cluster_normalize_scaler, cluster_y_scaler = create_scaled_data_by_col(train_df, min_max_cols, normalize_cols, y_cols, split_col, cluster)
    train_df_by_cluster[cluster] = cluster_df
    scaler_x_by_cluster[cluster] = (cluster_min_max_scaler, cluster_normalize_scaler)
    scaler_y_by_cluster[cluster] = cluster_y_scaler

if not final_run:
    val_df_by_cluster = {}

    for cluster in df[split_col].unique():
        val_cluster_min_max_scaler, val_cluster_normalize_scaler = scaler_x_by_cluster[cluster]
        val_cluster_y_scaler = scaler_y_by_cluster[cluster]

        val_cluster_df = val_df[val_df[split_col] == cluster]
        val_cluster_df = val_cluster_df.drop(columns=split_col)

        val_cluster_x_min_max = val_cluster_df[min_max_cols].values.astype(np.float32)
        val_cluster_x_normalize = val_cluster_df[normalize_cols].values.astype(np.float32)
        val_cluster_y = val_cluster_df[y_cols].values.reshape(-1, len(y_cols)).astype(np.float32)

        val_cluster_x_min_max = val_cluster_min_max_scaler.transform(val_cluster_x_min_max)
        val_cluster_x_normalize = val_cluster_normalize_scaler.transform(val_cluster_x_normalize)
        val_cluster_y = val_cluster_y_scaler.transform(val_cluster_y)

        val_cluster_df[min_max_cols] = val_cluster_x_min_max
        val_cluster_df[normalize_cols] = val_cluster_x_normalize
        val_cluster_df[y_cols] = val_cluster_y

        val_df_by_cluster[cluster] = val_cluster_df

['store_nbr', 'city', 'state', 'store_type', 'cluster', 'h_type_nat', 'h_description_nat', 'h_transferred_nat', 'h_type_loc', 'h_description_loc', 'h_transferred_loc', 'month', 'day', 'day_of_week', 'is_payday', 'is_earth_quake', 'is_weekday', 'is_weekend']
Index(['date', 'store_nbr', 'family', 'sales', 'onpromotion', 'city', 'state',
       'store_type', 'cluster', 'oil', 'h_type_nat', 'h_description_nat',
       'h_transferred_nat', 'h_type_loc', 'h_description_loc',
       'h_transferred_loc', 'transactions', 'year', 'month', 'day',
       'day_of_week', 'dow_avg_sales', 'dow_rolling_1_sales',
       'dow_rolling_3_sales', 'dow_rolling_7_sales', 'dow_avg_transactions',
       'dow_rolling_1_transactions', 'dow_rolling_3_transactions',
       'dow_rolling_7_transactions', 'prev_1_sales', 'prev_7_sales',
       'prev_14_sales', 'prev_1_transactions', 'prev_7_transactions',
       'prev_14_transactions', 'is_payday', 'is_earth_quake', 'is_weekday',
       'is_weekend'],
      dtype='ob

## NN Training

In [13]:
net_by_cluster = {}
train_params = {
                "criterion": nn.L1Loss,
                "optimizer": torch.optim.AdamW,
                "optimizer__weight_decay": 1e-8,
                #'train_split' : None,
                #"train_split": predefined_split(Dataset(val_x, val_y)),
                "lr": 0.001,
                "batch_size": 32,
                "max_epochs": 1000,
                "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
                "iterator_train__shuffle": False,
                "iterator_train__num_workers": 2,
                "iterator_train__pin_memory": True,
                "iterator_valid__shuffle": False,
                "iterator_valid__num_workers": 2,
                "iterator_valid__pin_memory": True,
                "verbose": 2,
        }

net_params = {
    'input_dim': len(x_cols),
    'out_dim': 1,
    'hidden_dim': 200,
    'num_hidden_layers': 6,
    }

In [5]:
for cluster in df[split_col].unique():
    train_df = train_df_by_cluster[cluster]
    train_x = train_df[x_cols].values.astype(np.float32)
    train_y = train_df[y_cols].values.reshape(-1, len(y_cols)).astype(np.float32)

    if not final_run:
        val_df = val_df_by_cluster[cluster]
        train_params['train_split'] = predefined_split(Dataset(val_df[x_cols].values.astype(np.float32), val_df[y_cols].values.reshape(-1, len(y_cols)).astype(np.float32)))
    else:
        train_params['train_split'] = None

    callbacks = [EarlyStopping(patience=10, threshold=0.0001, threshold_mode='abs', monitor='valid_loss', lower_is_better=True),
            Checkpoint(monitor='valid_loss_best', f_params=f'sales_forecaster_{cluster}.pt', dirname='models/'),
            LRScheduler(policy=ReduceLROnPlateau, monitor='train_loss', factor=0.5, patience=5, threshold=0.001, threshold_mode='abs', mode='min', verbose=True)
            ]

    train_params['callbacks'] = callbacks

    net = NeuralNetRegressor(FFNeuralNetwork(**net_params), **train_params)

    print(cluster)

     
    net.fit(train_x, train_y)
    net_by_cluster[cluster] = net

0


/home/codygrogan/.local/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.4307        0.4698     +  4.1980
      2        0.4219        0.4683     +  4.1347
      3        0.4196        0.4694        4.1510
      4        0.4174        0.4707        4.2521
      5        0.4155        0.4757        4.1395
      6        0.4136        0.4734        4.2411
      7        0.4125        0.4704        4.2442
      8        0.4107        0.4728        4.3170
      9        0.4091        0.4697        4.2836
     10        0.4077        0.4718        4.2592
     11        0.4062        0.4702        4.2804
Stopping since valid_loss has not improved in the last 10 epochs.
1


/home/codygrogan/.local/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.1678        0.2509     +  4.2360
      2        0.1549        0.2510        4.2950
      3        0.1538        0.2528        4.3367
      4        0.1528        0.2550        4.2269
      5        0.1528        0.2545        4.2620
      6        0.1528        0.2554        4.3451
      7        0.1519        0.2557        4.3433
      8        0.1506        0.2565        4.2788
      9        0.1512        0.2573        4.2632
     10        0.1500        0.2586        4.3233
Stopping since valid_loss has not improved in the last 10 epochs.
2


...

  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.1325        0.1160     +  4.2150
      2        0.1080        0.1113     +  4.1338
      3        0.0922        0.1096     +  4.0590
      4        0.0882        0.1110        4.1530
      5        0.0862        0.1101        4.0933
      6        0.0833        0.1109        4.0371
      7        0.0819        0.1106        4.0174
      8        0.0818        0.1118        4.0990
      9        0.0798        0.1114        4.1012
     10        0.0791        0.1078     +  4.1422
     11        0.0786        0.1104        4.1622
     12        0.0777        0.1102        4.1061
     13        0.0768        0.1103        4.1152
     14        0.0747        0.1094        4.2103
     15        0.0761        0.1138        4.1461
     16        0.0744        0.1140        4.0718
     17        0.0733        0.1180        4.1377
     18        0.0738        0.1093        4.1387


/home/codygrogan/.local/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.1016        1.3946     +  4.1521
      2        0.0903        1.2724     +  4.1336
      3        0.0876        1.2728        4.1086
      4        0.0861        1.3153        4.1654
      5        0.0841        1.3924        4.1199
      6        0.0847        1.5598        4.0842
      7        0.0821        1.2871        4.1251
      8        0.0824        1.4577        4.0982
      9        0.0830        1.6859        4.0939
     10        0.0811        1.4115        4.0695
     11        0.0816        1.5884        4.1599
Stopping since valid_loss has not improved in the last 10 epochs.
32


/home/codygrogan/.local/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


  epoch    train_loss    valid_loss    cp     dur
-------  ------------  ------------  ----  ------
      1        0.1733        0.1435     +  4.0772
      2        0.1621        0.1426     +  4.1007
      3        0.1592        0.1403     +  4.0873
      4        0.1577        0.1402     +  4.1215
      5        0.1565        0.1399     +  4.1222
      6        0.1549        0.1401        4.1017
      7        0.1540        0.1401        4.1105
      8        0.1529        0.1399        4.1057
      9        0.1522        0.1397     +  4.1780
     10        0.1515        0.1394     +  4.1811
     11        0.1513        0.1410        4.1203
     12        0.1504        0.1400        4.2257
     13        0.1498        0.1407        4.1253
     14        0.1493        0.1424        4.1368
     15        0.1489        0.1419        4.1304
     16        0.1490        0.1413        4.1956
     17        0.1486        0.1412        4.1645
     18        0.1479        0.1417        4.0550


# Load Nets from Checkpoints

In [14]:
for cluster in df[split_col].unique():
    net = NeuralNetRegressor(FFNeuralNetwork(**net_params), **train_params)
    net.initialize()
    net.load_params(f_params=f'models/sales_forecaster_{cluster}.pt')
    net_by_cluster[cluster] = net

## LSTM Training

In [23]:
endogenous_cols = [
        'sales', 'onpromotion', 'oil', 
       'dow_avg_sales', 'dow_rolling_3_sales', 'dow_rolling_7_sales',
       'dow_avg_transactions', 'dow_rolling_3_transactions',
       'dow_rolling_7_transactions', 'rolling_7_sales', 'rolling_14_sales',
       'rolling_7_transactions', 'rolling_14_transactions']

exogenous_cols = [
    'h_type_nat', 'h_description_nat', 'h_transferred_nat', 'h_type_loc',
    'h_description_loc', 'h_transferred_loc', 'month', 'day', 'day_of_week', 'store_nbr'
    ]

out_cols = ['sales']


lstm_net_by_cluster = {}
lstm_net_params = {
    'input_dim': 512,
    'endogenous_dim': len(endogenous_cols)*54,
    'endogenous_len': 5,
    'exogenous_dim': len(exogenous_cols),
    'hidden_dim': 1024,
    'out_dim': 54,
    'out_seq_len': 15,
    'num_layers': 4
}

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_epochs = 1000

for cluster in df[split_col].unique():
    train_df = train_df_by_cluster[cluster]
    train_dataset = LSTMDataset(train_df, 5, 15, 'date', endogenous_cols, exogenous_cols, out_cols, 'store_nbr')
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=30, shuffle=False)

    scaler_y = scaler_y_by_cluster[cluster]

    val_df = val_df_by_cluster[cluster]
    val_dataset = LSTMDataset(val_df, 5, 15, 'date', endogenous_cols, exogenous_cols, out_cols, 'store_nbr')

    model = LSTMNeuralNetwork(**lstm_net_params)
    optim = torch.optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-8)
    criterion = nn.L1Loss()
    model.train()
    model.to(device)
    model.zero_grad()

    val = val_dataset[0]
    val_endog = torch.tensor(val['endog']).unsqueeze(0).to(device).to(torch.float32)
    val_exog = torch.tensor(val['exog']).unsqueeze(0).to(device).to(torch.float32)
    val_y = scaler_y.inverse_transform(val['label'].reshape(-1,1).astype(np.float32))

    for epoch in range(num_epochs):
        for sample in train_loader:
            endog = sample['endog'].to(device).to(torch.float32)
            exog = sample['exog'].to(device).to(torch.float32)
            y = sample['label'].to(device).to(torch.float32)
            optim.zero_grad()
            output = model((endog, exog))
            loss = criterion(output, y)
            loss.backward()
            optim.step()
        with torch.no_grad():
            model.eval()
            val_pred = model((val_endog, val_exog))
            val_pred = val_pred.view(-1,1).cpu().detach().numpy()
            val_pred = scaler_y.inverse_transform(val_pred)

            val_loss1 = rmsle(val_y, val_pred.clip(0))
            val_loss2 = np.abs(val_y - val_pred).sum()
            print(f'Epoch {epoch+1}/{num_epochs}, RMSLE: {val_loss1.item():6.5f} L1: {val_loss2:6.5f}')
            model.train()


    lstm_net_by_cluster[cluster] = model

Epoch 1/1000, RMSLE: 0.59281 L1: 2621.91406
Epoch 2/1000, RMSLE: 0.59805 L1: 2591.96606
Epoch 3/1000, RMSLE: 0.61355 L1: 2627.24707
Epoch 4/1000, RMSLE: 0.61980 L1: 2648.24902
Epoch 5/1000, RMSLE: 0.63566 L1: 2669.86035
Epoch 6/1000, RMSLE: 0.64438 L1: 2688.27637
Epoch 7/1000, RMSLE: 0.66854 L1: 2746.78809
Epoch 8/1000, RMSLE: 0.68094 L1: 2764.41455
Epoch 9/1000, RMSLE: 0.69042 L1: 2781.10596
Epoch 10/1000, RMSLE: 0.69891 L1: 2787.39185
Epoch 11/1000, RMSLE: 0.69744 L1: 2791.86279
Epoch 12/1000, RMSLE: 0.69394 L1: 2789.51074
Epoch 13/1000, RMSLE: 0.70647 L1: 2797.06030
Epoch 14/1000, RMSLE: 0.69428 L1: 2787.90845
Epoch 15/1000, RMSLE: 0.70769 L1: 2804.49365
Epoch 16/1000, RMSLE: 0.69218 L1: 2788.97607
Epoch 17/1000, RMSLE: 0.71373 L1: 2809.92358
Epoch 18/1000, RMSLE: 0.68829 L1: 2788.63428
Epoch 19/1000, RMSLE: 0.71235 L1: 2810.70557
Epoch 20/1000, RMSLE: 0.68650 L1: 2790.86255
Epoch 21/1000, RMSLE: 0.71120 L1: 2808.48950


## Random Forest

In [15]:
from sklearn.ensemble import RandomForestRegressor
cluster_rfs = {}

for cluster in df[split_col].unique():
    train_df = train_df_by_cluster[cluster]

    train_x = train_df[x_cols].values.astype(np.float32)
    train_y = train_df[y_cols].values.reshape(-1, len(y_cols)).astype(np.float32)

    rf = RandomForestRegressor(n_estimators=100, max_depth=15, random_state=42, n_jobs=4)
    rf.fit(train_x, train_y.squeeze())

    cluster_rfs[cluster] = rf

## XGBoost

In [18]:
import xgboost as xgb
cluster_xgb = {}
for cluster in df[split_col].unique():
    train_x = train_x_by_cluster[cluster]
    train_y = train_y_by_cluster[cluster]

    xgb_model = xgb.XGBRegressor(n_estimators=1000, max_depth=12, learning_rate=0.001, random_state=42, n_jobs=2)
    xgb_model.fit(train_x, train_y.squeeze())

    cluster_xgb[cluster] = xgb_model

## Predict on Training Data

In [27]:
net_train_preds = []
rf_train_preds = []

for cluster in df[split_col].unique():
    train_df = train_df_by_cluster[cluster]

    train_x = train_df[x_cols].values.astype(np.float32)
    train_y = train_df[y_cols].values.reshape(-1, len(y_cols)).astype(np.float32)
    y_scaler = scaler_y_by_cluster[cluster]

    net = net_by_cluster[cluster]
    rf = cluster_rfs[cluster]

    net_preds = net.predict(train_x)
    rf_preds = rf.predict(train_x)

    train_df['sales_nn'] = net_preds
    train_df['sales_rf'] = rf_preds

## Validation Loss Evaluation

In [20]:
def rmsle(y_true, y_pred):
    return np.sqrt(np.mean(np.square(np.log1p(y_true) - np.log1p(y_pred))))

rf_preds = []
net_preds = []
xgb_preds = []
val_y_true = []

for cluster in df[split_col].unique():
    val_cluster_df = val_df_by_cluster[cluster]
    val_x = val_cluster_df[x_cols].values.astype(np.float32)
    val_y = val_cluster_df[y_cols].values.reshape(-1, len(y_cols)).astype(np.float32)

    rf = cluster_rfs[cluster]
    net = net_by_cluster[cluster]

    rf_preds.append(scaler_y_by_cluster[cluster].inverse_transform(rf.predict(val_x).reshape(-1, 1)))
    net_preds.append(scaler_y_by_cluster[cluster].inverse_transform(net.predict(val_x).reshape(-1, 1)).clip(0))    
    #xgb_preds.append(scaler_y_by_cluster[cluster].inverse_transform(cluster_xgb[cluster].predict(val_x).reshape(-1, 1)))
    val_y_true.append(scaler_y_by_cluster[cluster].inverse_transform(val_y))

    #rf_preds.append(rf.predict(val_x).reshape(-1, 1))
    #net_preds.append(net.predict(val_x).reshape(-1, 1).clip(0) )
    #val_y_true.append(val_y)   


rf_preds = np.concatenate(rf_preds)
net_preds = np.concatenate(net_preds)
#xgb_preds = np.concatenate(xgb_preds)
val_y_true = np.concatenate(val_y_true)

print(f'RF RMSLE: {rmsle(val_y_true, rf_preds)}')
#print(f'XGB RMSLE: {rmsle(val_y_true, xgb_preds)}')
print(f'NN RMSLE: {rmsle(val_y_true, net_preds)}')

RF RMSLE: 0.4077625548953881
NN RMSLE: 0.39476779103279114


## Loading Test Data

In [21]:
test_df = pd.read_csv(os.path.join(data_dir, 'test_data.csv'), index_col=0)
display(test_df.head())

test_x_by_cluster = {}
test_id_by_cluster = {}

for cluster in df[split_col].unique():
    test_cluster_min_max_scaler, test_cluster_normalize_scaler = scaler_x_by_cluster[cluster]
    test_cluster_y_scaler = scaler_y_by_cluster[cluster]

    test_cluster_x_df = test_df[test_df[split_col] == cluster]
    test_cluster_x_df = test_cluster_x_df.drop(columns=split_col)

    test_cluster_x_min_max = test_cluster_x_df[min_max_cols].values.astype(np.float32)
    test_cluster_x_normalize = test_cluster_x_df[normalize_cols].values.astype(np.float32)

    test_cluster_x_min_max = test_cluster_min_max_scaler.transform(test_cluster_x_min_max)
    test_cluster_x_normalize = test_cluster_normalize_scaler.transform(test_cluster_x_normalize)
    #test_cluster_x_normalize = test_cluster_x_normalize

    test_x_by_cluster[cluster] = np.concatenate([test_cluster_x_min_max, test_cluster_x_normalize], axis=1)
    test_id_by_cluster[cluster] = test_cluster_x_df.index


test_preds_dfs = []

for cluster in df[split_col].unique():
    test_x = test_x_by_cluster[cluster]
    id = test_id_by_cluster[cluster]
    rf = cluster_rfs[cluster]
    net = net_by_cluster[cluster]

    pred_rf = scaler_y_by_cluster[cluster].inverse_transform(rf.predict(test_x).reshape(-1, 1))
    #pred_xgb = scaler_y_by_cluster[cluster].inverse_transform(cluster_xgb[cluster].predict(test_x).reshape(-1, 1))
    pred_nn = scaler_y_by_cluster[cluster].inverse_transform(net_by_cluster[cluster].predict(test_x).reshape(-1, 1)).clip(0)

    #pred_rf = rf.predict(test_x).reshape(-1, 1)
    #pred_nn = net.predict(test_x).reshape(-1, 1).clip(0)

    
    #cluster_df = pd.DataFrame(np.concatenate([pred_nn], axis=1), index=id, columns=['sales_nn'])
    cluster_df = pd.DataFrame(np.concatenate([pred_rf, pred_nn], axis=1), index=id, columns=['sales_rf', 'sales_nn'])

    test_preds_dfs.append(cluster_df)

test_preds_df = pd.concat(test_preds_dfs)

test_df = test_df.merge(test_preds_df, on='id', how='left')

sub_df_nn = test_df[['sales_nn']]
sub_df_rf = test_df[['sales_rf']]
#sub_df_xgb = test_df[['sales_xgb']]

sub_df_rf = sub_df_rf.rename(columns={'sales_rf': 'sales'})
#sub_df_xgb = sub_df_xgb.rename(columns={'sales_xgb': 'sales'})
sub_df_nn = sub_df_nn.rename(columns={'sales_nn': 'sales'})


display(sub_df_nn.head())
display(sub_df_rf.head())
#display(sub_df_xgb.head())

sub_df_nn.to_csv('data/submission_nn.csv')
#sub_df_xgb.to_csv('data/submission_xgb.csv')
sub_df_rf.to_csv('data/submission_rf.csv')

,store_nbr,family,onpromotion,city,state,store_type,cluster,oil,h_type_nat,h_description_nat,...,dow_rolling_1_transactions,dow_rolling_3_transactions,dow_rolling_7_transactions,prev_1_transactions,prev_7_transactions,prev_14_transactions,is_payday,is_earth_quake,is_weekday,is_weekend
id,,,,,,,,,,,,,,,,,,,,,
3000888,1,0,0,0,0,0,13,46.8,0,0,...,1892.0,1864.0,1888.857143,1766.0,1892.0,1903.0,False,False,1,0
3000889,1,1,0,0,0,0,13,46.8,0,0,...,1892.0,1864.0,1888.857143,1766.0,1892.0,1903.0,False,False,1,0
3000890,1,2,2,0,0,0,13,46.8,0,0,...,1892.0,1864.0,1888.857143,1766.0,1892.0,1903.0,False,False,1,0
3000891,1,3,20,0,0,0,13,46.8,0,0,...,1892.0,1864.0,1888.857143,1766.0,1892.0,1903.0,False,False,1,0
3000892,1,4,0,0,0,0,13,46.8,0,0,...,1892.0,1864.0,1888.857143,1766.0,1892.0,1903.0,False,False,1,0


,sales
id,
3000888,4.415288
3000889,0.000120
3000890,4.730282
3000891,2429.969482
3000892,0.000000


,sales
id,
3000888,4.038700
3000889,0.004688
3000890,4.437217
3000891,2406.571371
3000892,0.166050
